In [1]:
import iris
import glob
import iris.plot as iplt
import iris.quickplot as qplt
import datetime as datetime
import iris.coord_categorisation as cat

In [2]:
year =2006

### Get list of files to convert

In [59]:
radardir = f'/nfs/a319/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
file_list=glob.glob(radardir +"*.nc")
sorted_list = sorted(file_list)

### Load radar data for one day (using IRIS)

In [66]:
sorted_list[84]

'/nfs/a319/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060829.nc'

In [60]:
day_cube = iris.load_cube(sorted_list[84])
day_cube

Rain Rate Composite (mm/hr),time,projection_y_coordinate,projection_x_coordinate
Shape,286,72,82
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
forecast_reference_time,x,-,-
Scalar coordinates,,,forecast_period 0 second
Attributes,,,Conventions CF-1.7field_code 213institution Met Officenimrod_version 2probability_period_of_event 0source 1km Single Site Radarstitle Unknown


### Add additional time based variables

In [61]:
# cat.add_year(day_cube, 'time', name='year')
# cat.add_month(day_cube, 'time', name='month')
# cat.add_day_of_month(day_cube, 'time', name='day_of_month')
cat.add_hour(day_cube, 'time', name='hour')
# cat.add_day(day_cube, 'time', name='day')

In [26]:
#day_cube.coord('month')

### Aggregate to hourly values (means)

In [174]:
# hourly_mean = day_cube.aggregated_by(['hour'],iris.analysis.MEAN)
# hourly_mean

### Aggregate to half hourly values (means)

In [62]:
firsthalfof_hour_constraint = iris.Constraint(time=lambda cell: cell.point.minute <30)
secondhalfof_hour_constraint = iris.Constraint(time=lambda cell: cell.point.minute >=30)

In [63]:
times = hour_cube.coord('time').points
times = [datetime.datetime.fromtimestamp(x ) for x in times]
times

[datetime.datetime(2006, 6, 28, 0, 0),
 datetime.datetime(2006, 6, 28, 0, 5),
 datetime.datetime(2006, 6, 28, 0, 10),
 datetime.datetime(2006, 6, 28, 0, 15),
 datetime.datetime(2006, 6, 28, 0, 20),
 datetime.datetime(2006, 6, 28, 0, 25),
 datetime.datetime(2006, 6, 28, 0, 30),
 datetime.datetime(2006, 6, 28, 0, 35),
 datetime.datetime(2006, 6, 28, 0, 40),
 datetime.datetime(2006, 6, 28, 0, 45),
 datetime.datetime(2006, 6, 28, 0, 50),
 datetime.datetime(2006, 6, 28, 0, 55)]

In [64]:
# for i in range(0,len(sorted_list)):
#     print(i)

#     ### Load radar data for one day (using IRIS)
#     day_cube = iris.load_cube(sorted_list[i])

In [65]:
# Create empty cube list to populate
my_cube_list = iris.cube.CubeList()

# Get list of the hours
hours = set(day_cube.coord('hour').points)
# Loop through the hours
for hour in hours:
    # Establish constraint to select only this hour
    hour_constraint = iris.Constraint(time=lambda cell: cell.point.hour == hour)
    # Use constraint to select only this hour
    hour_cube = day_cube.extract(hour_constraint)
    # Check the times
    # times = hour_cube.coord('time').points
    # times = [datetime.datetime.fromtimestamp(x ) for x in times]
    
    # Get only cubes which fall within the first half of the hour and then the second half of the hour
    first_half_of_hour = hour_cube.extract(firsthalfof_hour_constraint)
    second_half_of_hour = hour_cube.extract(secondhalfof_hour_constraint)
    
    # If there are at least 4 values
    # Find the mean across first/second halves of hour
    # Add to cube list
    if first_half_of_hour == None:
        print("no values in 1st half hour")
    elif len(first_half_of_hour.shape) ==2:
        print("only 1 value in 2nd half hour")        
    else:
        if first_half_of_hour.shape[0] >=4:
            first_half_hourly_mean = first_half_of_hour.aggregated_by(['hour'],iris.analysis.MEAN)
            my_cube_list.append(first_half_hourly_mean)
        else:
            print(f"only {first_half_of_hour.shape[0]} vals in 1st half hour")
    
    ### SECOND HALF HOUR    
    if second_half_of_hour == None:
        print("no values in 2nd half hour")
    elif len(second_half_of_hour.shape) ==2:
        print("only 1 value in 2nd half hour")
    else:
        if second_half_of_hour.shape[0] >=4:    
            second_half_hourly_mean = second_half_of_hour.aggregated_by(['hour'],iris.analysis.MEAN)
            my_cube_list.append(second_half_hourly_mean)
        else:
            print(f"only {second_half_of_hour.shape[0]} vals in 2nd half hour")

### Join back into one cube covering the whole day

In [94]:
thirty_mins_means = my_cube_list.concatenate_cube()

In [62]:
# times = thirty_mins_means.coord('time').points
# times = [datetime.datetime.fromtimestamp(x ) for x in times]

In [95]:
new_fp = sorted_list[0][:-3]+ '_30mins.nc'
new_fp = new_fp.replace('5mins', '30mins')
new_fp

'/nfs/a319/gy17m2a/PhD/datadir/NimRod/30mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060601_30mins.nc'

In [96]:
thirty_mins_means

Rain Rate Composite (mm/hr),time,projection_y_coordinate,projection_x_coordinate
Shape,48,72,82
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
forecast_reference_time,x,-,-
hour,x,-,-
Scalar coordinates,,,forecast_period 0 second
Cell methods,,,


In [97]:
iris.save(thirty_mins_means, new_fp)